In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [11]:
import pandas as pd
from Data.data_retrieval import get_data
import Feature_Selection
from tqdm import tqdm

In [14]:
def correlation_filter(features_df,number_of_filtered_selections, ohlcv):
    selected_features = []
    while len(selected_features) < number_of_filtered_selections:
        chunksize=10
        df = pd.DataFrame()
        for row in tqdm(range(chunksize)):
            f = getattr(Feature_Selection, features_df.iloc[row]["F"])
            flb = features_df.iloc[row]["F_Lookback"]
            g = getattr(Feature_Selection, features_df.iloc[row]["G"])
            glb = features_df.iloc[row]["G_Lookback"]
            X = f([ohlcv, flb]).iloc[flb - 1:].reset_index(drop=True)
            col_name = X.columns[-1]
            X = g(X, col_name, glb).iloc[glb - 1:].reset_index(drop=True)
            X = X[[X.columns[-1]]].set_index(X["Datetime"])
            df = pd.concat([df, X], axis=1)

        df1 = df.copy()
        df.dropna(inplace=True)
        df = df.loc[:,~df.columns.duplicated()]
        corr_mat = df.corr()
        features = [column for column in df]
        if len(selected_features)==0:
            selected_features.append(features[0])
            features.remove(features[0])
            last_selected_feature = features[0]
        else:
            last_selected_feature = selected_features[-1]

        while len(selected_features) < number_of_filtered_selections:
            corrs = corr_mat.loc[features][last_selected_feature]
            corrs = corrs.loc[corrs < 0.999]
            features = [f for f in features if f not in corrs.index.to_list()]
            if len(features) == 0:
                break
            feat = features[0]
            selected_features.append(feat)
            features.remove(feat)
            last_selected_feature = feat

        indices = []
        for feat in selected_features:
            indices.append(list(df1.columns).index(feat))
        selected_features = features_df.iloc[indices].reset_index(drop=True)
        return selected_features

In [ ]:
df = pd.read_csv("FeatureSelectionResults.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
ohlcv = get_data('NIFc1', 'D')
number_of_filtered_selections = 100
filtered_strategies = correlation_filter(df,number_of_filtered_selections, ohlcv)
filtered_strategies.to_csv("FilteredFeatureSelectionResults.csv")
print(len(filtered_strategies))

In [5]:
df = pd.read_csv("FeatureSelectionResults.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
ohlcv = get_data('NIFc1', 'D')

In [15]:
number_of_filtered_selections = 100
filtered_strategies = correlation_filter(df,number_of_filtered_selections, ohlcv)

100%|██████████| 10000/10000 [09:00<00:00, 18.50it/s]


In [18]:
filtered_strategies.dropna().corr()

KeyboardInterrupt: 